## Работа с YandexGPT и OpenSearch с помощью фреймворка Langchain

### 1. Устанавливаем необходимые библиотеки

In [1]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 2.5 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.24.0 requires pympler<2,>=0.9, which is not installed.
ml-kernel 0.0.1 requires urllib3==1.25.11, but you have urllib3 2.2.1 which is incompatible.
argilla 1.16.0 requires typer<0.8.0,>=0.6.0, but you have typer 0.9.0 which is incompatible.
chromadb 0.4.24 requires numpy>=1.22.5, but you have numpy 1.22.4 which is incompatible.
streamlit 1.24.0 requires importlib-metadata<7,>=1.4, but you have importlib-metadata 7.1.0 which is incompatible.
yandex-speechkit 1.3.3 requires grpcio<=1.46.3, but you have grpcio 1.62.2 which is incompatible.
yandex-speechkit 1.3.3 re

После установки библиотек необходимо перегазгрузить Kernel. Для этого в верхнем меню выберите Kernel -> Restart Kernel

### 2. Подключение к кластеру OpenSearch

Документация по подключению: https://cloud.yandex.ru/docs/managed-opensearch/operations/connect

#### Получение SSL-сертификата

In [2]:
!mkdir -p /home/jupyter/datasphere/project/.opensearch && \
wget "https://storage.yandexcloud.net/cloud-certs/CA.pem" \
     --output-document /home/jupyter/datasphere/project/.opensearch/root.crt && \
chmod 0600 /home/jupyter/datasphere/project/.opensearch/root.crt

--2024-04-25 10:55:11--  https://storage.yandexcloud.net/cloud-certs/CA.pem
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3579 (3.5K) [application/x-x509-ca-cert]
Saving to: ‘/home/jupyter/datasphere/project/.opensearch/root.crt’

     0K ...                                                   100% 4.43G=0s

2024-04-25 10:55:11 (4.43 GB/s) - ‘/home/jupyter/datasphere/project/.opensearch/root.crt’ saved [3579/3579]



#### Тестируем подключение к кластеру

In [3]:
CA = '/home/jupyter/datasphere/project/.opensearch/root.crt'

import os
#Укажите пароль для пользователя admin
PASS = os.environ['MDB_OS_PWD'] # Обратиться к секрету <имя_секрета> из переменных окружения
#Укажите список хостов кластера
HOSTS = os.environ['MDB_OS_HOSTS'].split(",")

In [4]:
from opensearchpy import OpenSearch

conn = OpenSearch(
  HOSTS,
  http_auth=('admin', PASS),
  use_ssl=True,
  verify_certs=True,
  ca_certs=CA)

print(conn.info())

{'name': 'rc1a-9r5h0inf2p0f3tk2.mdb.yandexcloud.net', 'cluster_name': 'c9qthb5rf6to08jrn7uo', 'cluster_uuid': 'us1xtR19RCe9tSmOydAvOg', 'version': {'distribution': 'opensearch', 'number': '2.8.0', 'build_type': 'deb', 'build_hash': '8cbab9609696cd93c45fd5f3090560648c04f5af', 'build_date': '2023-10-04T14:42:52.695597332Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.10.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'The OpenSearch Project: https://opensearch.org/'}


При правильной настройке подключения код выше вернет объект dict с информацией об имени кластера, uuid, версии и т.д.

### 2. Получаем IAM-токен для работы с YandexGPT

In [5]:
import time
import jwt
import requests

Для того, чтобы обратиться из DataSphere к YandexGPT в настройках проекта необходимо указать сервисный аккаунт, у которого есть роль ai.languageModels.user. Для сервисного аккаунта необходимо создать **авторизованный** ключ доступа. 

In [6]:
service_account_id = os.environ["SA_ID"]
key_id = os.environ["KEY_ID"]
# Как получить private key: https://yandex.cloud/ru/docs/iam/operations/iam-token/create-for-sa
private_key = os.environ['PRIVATE_KEY'].replace('\\n', '\n')

In [7]:
# Получаем IAM-токен

now = int(time.time())
payload = {
        'aud': 'https://iam.api.cloud.yandex.net/iam/v1/tokens',
        'iss': service_account_id,
        'iat': now,
        'exp': now + 360}

# Формирование JWT
encoded_token = jwt.encode(
    payload,
    private_key,
    algorithm='PS256',
    headers={'kid': key_id})

url = 'https://iam.api.cloud.yandex.net/iam/v1/tokens'
x = requests.post(url,  
                  headers={'Content-Type': 'application/json'},
                  json = {'jwt': encoded_token}).json()
token = x['iamToken']

### 3. Создаем индекс в OpenSearch для поиска релевантных документов

Устанавливаем LangChain. **LangChain** — это фреймворк для разработки приложений на базе языковых моделей.

In [9]:
import langchain
from operator import itemgetter
from langchain.document_loaders import DirectoryLoader, PyPDFLoader, TextLoader, UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [11]:
# Указываем путь к папке с документами в Object Storage. Для этого выберите Сopy Path
source_dir = "/home/jupyter/datasphere/s3/ds-knights-demo"
# source_dir = "/home/jupyter/datasphere/s3/dz-yc-docs/yc-docs-2024-q2-md/ru/"

In [11]:
# Создаем объект для считывания документов из бакета в S3
loader = langchain.document_loaders.DirectoryLoader(source_dir, 
                                                    glob="**/*.md",
                                                    silent_errors=True,
                                                    show_progress=True, 
                                                    recursive=True,
                                                    use_multithreading=True,
                                                    loader_cls=UnstructuredMarkdownLoader)

In [12]:
# Создаем объект для считывания документов из S3 бакета 
loader = DirectoryLoader(source_dir, 
                         glob="**/*.pdf", 
                         loader_cls=PyPDFLoader, 
                         silent_errors=True,
                         show_progress=True, 
                         recursive=True
)

In [17]:
# Создаем объект для считывания документов из бакета в S3
# loader = DirectoryLoader(source_dir, 
#                         glob="**/*.txt",
#                         silent_errors=True,
#                         show_progress=True, 
#                         recursive=True,
#                        loader_cls=TextLoader)

In [13]:
# Указываем длины фрагмента, на который разбиваются документы

CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

In [14]:
documents = loader.load()

100%|██████████| 2/2 [00:01<00:00,  1.43it/s]


In [15]:
len(documents)

7

In [16]:
# Считываем документы и разбиваем на фрагменты

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE, 
    chunk_overlap=CHUNK_OVERLAP
)
docs = text_splitter.split_documents(documents)

print(f"Количество фрагментов, на которые разбиты все документы = {len(docs)}")

Количество фрагментов, на которые разбиты все документы = 18


In [ ]:
# md_splitter = RecursiveCharacterTextSplitter.from_language(
#     language=Language.MARKDOWN, chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP
# )
# md_docs = md_splitter.create_documents()

In [17]:
# Cоздаем объект YandexGPTEmbeddings для построения векторов с помощью YandexGPT
from yandex_chain import YandexEmbeddings
folder_id = os.environ["YC_FOLDER_ID"]
embeddings = YandexEmbeddings(folder_id=folder_id, iam_token = token)
embeddings.sleep_interval = 0.02 #текущее ограничение эмбеддера 50 RPS, делаем задержку 1/50 секунды, чтобы не выйти за это ограничение

In [18]:
text_to_print = f"Ориентировочное время оцифровки = {len(documents)*embeddings.sleep_interval} с."
print(text_to_print)

Ориентировочное время оцифровки = 0.14 с.


In [19]:
# Строим вектора по документам и добавляем их в базу OpenSearch
from langchain_community.vectorstores import OpenSearchVectorSearch

docsearch = OpenSearchVectorSearch.from_documents(
     docs,
     embeddings,
     opensearch_url=HOSTS,
     http_auth=("admin", PASS),
     use_ssl = True,
     verify_certs = True,
     ca_certs = CA,
     engine = 'lucene',
     index_name = "dz-network-1000-200",
     bulk_size=1000000,
     space_type="cosinesimil"
)
# space_type: "l2", "l1", "cosinesimil", "linf", "innerproduct"; default: "l2"

In [20]:
# Если эмбеддинги уже есть, то запускаем эту строчку
docsearch2 = OpenSearchVectorSearch (
    embedding_function=embeddings,
    # index_name = "yc-docs-md-1000-200",
    index_name = "ycdoc-24q2-1000-200-md",
    opensearch_url=HOSTS,
    http_auth=("admin", PASS),
    use_ssl = True,
    verify_certs = True,
    ca_certs = CA,
    engine = 'lucene'
)  

In [21]:
# Тестируем сохраненные вектора в docssearch
query = "Как устроена сеть в облаке?"
docs = docsearch.similarity_search(query, k=2)
docs

[Document(page_content='13.02.2024, 16:08 Устройство сети в Yandex Cloud | Yandex Cloud - Документация\nhttps://cloud.yandex.ru/ru/docs/vpc/concepts/network-overview 2/5\nОдин из основных компонентов физической сети — транспортная сеть Yandex Cloud.\nК транспортной сети подключаются:\nЦОД (зоны доступности) Yande x Cloud.\nТочки присутствия (Point of Presence или PoP). На точках прис утствия размещается сетевое оборудование транспортной сети. На отдельных точка\nвнешним се тям и  интернету (Internet Peering). На отдельных точках присутствия клиентам Yande x Cloud предоставляется возможность организова\nсобственной инфраструктуре и облачными ресурсами  и публичным се рвисам  Yande x Cloud с п омощью услуги Yande x Cloud Interconne ct.\nТранспортная сеть Yande x Cloud со стоит из двух уровней:\nУровень оптической сети DWD M. Оборудование оптического спектрального уплотнения (DWD M ) обеспечивает подключение оборудования уро', metadata={'source': '/home/jupyter/datasphere/s3/ds-knights-de

In [22]:
# Тестируем сохраненные вектора в docssearch2
query = "Как устроена сеть в облаке?"
docs2 = docsearch2.similarity_search(query, k=2)
docs2

[Document(page_content='description: "Облачная сеть — это аналог традиционной локальной сети в дата-центре. Облачные сети создаются в каталогах и используются для передачи информации между облачными ресурсами и связи ресурсов с интернетом."\nkeywords:\n  - облачная сеть\n  - облако\n  - сеть\n\nОблачные сети и подсети\n\nОблачные сети {#network}\n\nОблачная сеть — это аналог традиционной локальной сети в дата-центре. Облачные сети создаются в каталогах и используются для передачи информации между облачными ресурсами и связи ресурсов с интернетом.\n\nДля виртуальных машин Yandex Compute Cloud и хостов баз данных доступ из интернета и в интернет открыт через публичные IP-адреса.\n\nУправлять трафиком в облачных сетях и подсетях можно с помощью групп безопасности. Они содержат правила, которые определяют протоколы и IP-адреса для приема и отправки трафика.\n\nПодсети {#subnet}', metadata={'source': '/home/jupyter/datasphere/s3/dz-yc-docs/yc-docs-2024-q2-md/ru/vpc/concepts/network.md'}),
 

In [23]:
# Поисковая выдача N1 по тематическому фрагменту документации
print(docs[0].page_content)

13.02.2024, 16:08 Устройство сети в Yandex Cloud | Yandex Cloud - Документация
https://cloud.yandex.ru/ru/docs/vpc/concepts/network-overview 2/5
Один из основных компонентов физической сети — транспортная сеть Yandex Cloud.
К транспортной сети подключаются:
ЦОД (зоны доступности) Yande x Cloud.
Точки присутствия (Point of Presence или PoP). На точках прис утствия размещается сетевое оборудование транспортной сети. На отдельных точка
внешним се тям и  интернету (Internet Peering). На отдельных точках присутствия клиентам Yande x Cloud предоставляется возможность организова
собственной инфраструктуре и облачными ресурсами  и публичным се рвисам  Yande x Cloud с п омощью услуги Yande x Cloud Interconne ct.
Транспортная сеть Yande x Cloud со стоит из двух уровней:
Уровень оптической сети DWD M. Оборудование оптического спектрального уплотнения (DWD M ) обеспечивает подключение оборудования уро


In [24]:
# Поисковая выдача N1 по большой документации
print(docs2[0].page_content)

description: "Облачная сеть — это аналог традиционной локальной сети в дата-центре. Облачные сети создаются в каталогах и используются для передачи информации между облачными ресурсами и связи ресурсов с интернетом."
keywords:
  - облачная сеть
  - облако
  - сеть

Облачные сети и подсети

Облачные сети {#network}

Облачная сеть — это аналог традиционной локальной сети в дата-центре. Облачные сети создаются в каталогах и используются для передачи информации между облачными ресурсами и связи ресурсов с интернетом.

Для виртуальных машин Yandex Compute Cloud и хостов баз данных доступ из интернета и в интернет открыт через публичные IP-адреса.

Управлять трафиком в облачных сетях и подсетях можно с помощью групп безопасности. Они содержат правила, которые определяют протоколы и IP-адреса для приема и отправки трафика.

Подсети {#subnet}


In [25]:
# В дальнейшем нам понадобится не similarity search метод, а метод retriever
# Сначала по тематическому фрагменту документации
retriever = docsearch.as_retriever(search_kwargs={"k": 2})
docs = retriever.invoke(query)
docs

[Document(page_content='13.02.2024, 16:08 Устройство сети в Yandex Cloud | Yandex Cloud - Документация\nhttps://cloud.yandex.ru/ru/docs/vpc/concepts/network-overview 2/5\nОдин из основных компонентов физической сети — транспортная сеть Yandex Cloud.\nК транспортной сети подключаются:\nЦОД (зоны доступности) Yande x Cloud.\nТочки присутствия (Point of Presence или PoP). На точках прис утствия размещается сетевое оборудование транспортной сети. На отдельных точка\nвнешним се тям и  интернету (Internet Peering). На отдельных точках присутствия клиентам Yande x Cloud предоставляется возможность организова\nсобственной инфраструктуре и облачными ресурсами  и публичным се рвисам  Yande x Cloud с п омощью услуги Yande x Cloud Interconne ct.\nТранспортная сеть Yande x Cloud со стоит из двух уровней:\nУровень оптической сети DWD M. Оборудование оптического спектрального уплотнения (DWD M ) обеспечивает подключение оборудования уро', metadata={'source': '/home/jupyter/datasphere/s3/ds-knights-de

In [26]:
# Теперь используем ретривер по большой документации
retriever2=docsearch2.as_retriever(search_kwargs={"k": 2})
docs2 = retriever2.invoke(query)
docs2

[Document(page_content='description: "Облачная сеть — это аналог традиционной локальной сети в дата-центре. Облачные сети создаются в каталогах и используются для передачи информации между облачными ресурсами и связи ресурсов с интернетом."\nkeywords:\n  - облачная сеть\n  - облако\n  - сеть\n\nОблачные сети и подсети\n\nОблачные сети {#network}\n\nОблачная сеть — это аналог традиционной локальной сети в дата-центре. Облачные сети создаются в каталогах и используются для передачи информации между облачными ресурсами и связи ресурсов с интернетом.\n\nДля виртуальных машин Yandex Compute Cloud и хостов баз данных доступ из интернета и в интернет открыт через публичные IP-адреса.\n\nУправлять трафиком в облачных сетях и подсетях можно с помощью групп безопасности. Они содержат правила, которые определяют протоколы и IP-адреса для приема и отправки трафика.\n\nПодсети {#subnet}', metadata={'source': '/home/jupyter/datasphere/s3/dz-yc-docs/yc-docs-2024-q2-md/ru/vpc/concepts/network.md'}),
 

In [27]:
# Чуть более читаемый вывод результатов поиска по тематическому фрагменту документации
for i in range(2):
    print(docs[i].page_content)

13.02.2024, 16:08 Устройство сети в Yandex Cloud | Yandex Cloud - Документация
https://cloud.yandex.ru/ru/docs/vpc/concepts/network-overview 2/5
Один из основных компонентов физической сети — транспортная сеть Yandex Cloud.
К транспортной сети подключаются:
ЦОД (зоны доступности) Yande x Cloud.
Точки присутствия (Point of Presence или PoP). На точках прис утствия размещается сетевое оборудование транспортной сети. На отдельных точка
внешним се тям и  интернету (Internet Peering). На отдельных точках присутствия клиентам Yande x Cloud предоставляется возможность организова
собственной инфраструктуре и облачными ресурсами  и публичным се рвисам  Yande x Cloud с п омощью услуги Yande x Cloud Interconne ct.
Транспортная сеть Yande x Cloud со стоит из двух уровней:
Уровень оптической сети DWD M. Оборудование оптического спектрального уплотнения (DWD M ) обеспечивает подключение оборудования уро
Доступ облачных ресурсо в к интернету.
Виртуальную сеть часто называют термином Overlay. В одном 

In [28]:
# Чуть более читаемый вывод результатов поиска по большой документации
for i in range(2):
    print(docs2[i].page_content)

description: "Облачная сеть — это аналог традиционной локальной сети в дата-центре. Облачные сети создаются в каталогах и используются для передачи информации между облачными ресурсами и связи ресурсов с интернетом."
keywords:
  - облачная сеть
  - облако
  - сеть

Облачные сети и подсети

Облачные сети {#network}

Облачная сеть — это аналог традиционной локальной сети в дата-центре. Облачные сети создаются в каталогах и используются для передачи информации между облачными ресурсами и связи ресурсов с интернетом.

Для виртуальных машин Yandex Compute Cloud и хостов баз данных доступ из интернета и в интернет открыт через публичные IP-адреса.

Управлять трафиком в облачных сетях и подсетях можно с помощью групп безопасности. Они содержат правила, которые определяют протоколы и IP-адреса для приема и отправки трафика.

Подсети {#subnet}
Устройство сети в Yandex Cloud

Сеть в Yandex Cloud можно условно разделить на две большие части:

Физическая сеть — это аппаратная сеть внутри центров о

### 4. Обращаемся при помощи LLM к выбранным документам

In [29]:
# Сначала определим шаблоны промптов для обработки документов и для языковой модели с учетом истории общения

from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_models import ChatYandexGPT


In [30]:
# выбираем какую yandexgpt модель будем использовать, и выставляем ее temperature = 0
# model_uri = "gpt://"+str(folder_id)+"/yandexgpt-lite/latest"
model_uri = "gpt://"+str(folder_id)+"/yandexgpt/latest"
llm = ChatYandexGPT(iam_token = token, model_uri=model_uri, temperature = 0, max_tokens=8000)

In [31]:
### Контекстуализированный вопрос ###
contextualize_q_system_prompt = """Учитывая историю общения и последний вопрос пользователя, \
который может ссылаться на ДОКУМЕНТЫ в истории чата, сформулируй отдельный самодостаточный вопрос, \
который можно понять без истории общения. НЕ отвечай на вопрос, \
просто переформулируй его, если необходимо, в противном случае верни его как есть."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [32]:
### Ответ на вопрос ###
qa_system_prompt = """Ты помощник, который отвечает на вопросы \
Используй ДОКУМЕНТЫ ниже, чтобы ответить на этот вопрос. \
Если ты не знаешь ответ, просто скажи, что не знаешь. \
Используй максимум 20 предложений и старайся, чтобы ответ был максимально подробным.\

ДОКУМЕНТЫ: {context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [49]:
history_aware_retriever = create_history_aware_retriever(
    llm, retriever2, contextualize_q_prompt
)

In [50]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [51]:
### Управление историей общения ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [52]:
inputs = {"input": "Как устроена сеть в облаке?"}

In [61]:
ai = conversational_rag_chain.invoke(
    inputs,
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)

In [62]:
ai["answer"]

'Сеть в облаке представляет собой сложную систему, которая обеспечивает взаимодействие между различными облачными ресурсами и сервисами. Она состоит из нескольких ключевых компонентов:\n\n1. **Облачные ресурсы:** Это могут быть виртуальные машины, контейнеры, базы данных и другие сервисы, которые предоставляются пользователям в облачной среде.\n\n2. **Виртуальные сети:** Облачные провайдеры создают виртуальные сети, которые позволяют пользователям объединять свои ресурсы и обеспечивать их взаимодействие. Виртуальные сети изолированы друг от друга, что обеспечивает безопасность и контроль над трафиком.\n\n3. **Подсети:** Внутри виртуальных сетей можно создавать подсети, которые обеспечивают дополнительную изоляцию и гибкость в управлении трафиком и безопасностью.\n\n4. **Группы безопасности:** Пользователи могут создавать группы безопасности, чтобы контролировать доступ к своим ресурсам и определять правила для входящего и исходящего трафика.\n\n5. **Маршрутизация:** Облачные платформы 

In [46]:
from langchain_core.messages import HumanMessage

chat_history = []

question = "Что такое облачная сеть?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])
print(ai_msg_1["answer"])
print("\n\n ******************************************** ")
second_question = "Как она устроена?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})
print(ai_msg_2["answer"])

Облачная сеть — это инфраструктура, которая обеспечивает взаимодействие между облачными ресурсами и интернетом. Она состоит из двух основных частей: физической и виртуальной сети.

Физическая сеть представляет собой аппаратную сеть внутри центров обработки данных (ЦОД) и транспортную сеть между ЦОД. Эта сеть обеспечивает базовую функциональность и называется Underlay.

Виртуальная сеть работает поверх физической сетевой инфраструктуры и предоставляет IP-связность между облачными сервисами. Она называется Overlay и может включать в себя несколько виртуальных сетей, созданных в одном каталоге ресурсов.

Для получения более подробной информации о физической и виртуальной сетях в Yandex Cloud рекомендуется обратиться к документации.


 ******************************************** 
Облачная сеть в Yandex Cloud состоит из двух основных частей: физической и виртуальной сети.

Физическая сеть представляет собой аппаратную сеть внутри центров обработки данных (ЦОД) и транспортную сеть между ЦОД

In [48]:
for document in ai_msg_2["context"]:
    print(document)
    print()

page_content='13.02.2024, 16:08 Устройство сети в Yandex Cloud | Yandex Cloud - Документация\nhttps://cloud.yandex.ru/ru/docs/vpc/concepts/network-overview 1/5Концепции\xa0/ Устройство сети в Yandex Cloud\nСтатья создана Yandex Cloud Обновлена \x00\x00 декабря \x00\x00\x00\x00 г.\nФизическая сеть в Yande x Cloud\nВиртуальная сеть в Yande x Cloud\nVRouter\nVRouter-agent\nCloudGate\nВиды сетевого взаимодействия\nОграничения\nДополнительные материалы\nСеть в Yande x Cloud мо жно условно разделить на две большие части:\nФизическая сеть — э то аппаратная сеть внутри центров обработки данных (ЦОД), транспортная сеть между ЦОД и в местах подключения к внешн\nназывают терми ном Underlay.\nВиртуальная сеть — р аботает поверх физической сетевой инфраструктуры. Сервисы виртуальной сети Virtual Private Cloud (VPC) предоставляют п\nIP-связность между облачными ресурса ми.\nДоступ облачных ресурсо в к интернету.\nВиртуальную сеть часто называют термином Overlay. В одном каталоге ресурсо в может быть

In [63]:
for document in ai["context"]:
    print(document)
    print()

page_content='description: "Облачная сеть — это аналог традиционной локальной сети в дата-центре. Облачные сети создаются в каталогах и используются для передачи информации между облачными ресурсами и связи ресурсов с интернетом."\nkeywords:\n  - облачная сеть\n  - облако\n  - сеть\n\nОблачные сети и подсети\n\nОблачные сети {#network}\n\nОблачная сеть — это аналог традиционной локальной сети в дата-центре. Облачные сети создаются в каталогах и используются для передачи информации между облачными ресурсами и связи ресурсов с интернетом.\n\nДля виртуальных машин Yandex Compute Cloud и хостов баз данных доступ из интернета и в интернет открыт через публичные IP-адреса.\n\nУправлять трафиком в облачных сетях и подсетях можно с помощью групп безопасности. Они содержат правила, которые определяют протоколы и IP-адреса для приема и отправки трафика.\n\nПодсети {#subnet}' metadata={'source': '/home/jupyter/datasphere/s3/dz-yc-docs/yc-docs-2024-q2-md/ru/vpc/concepts/network.md'}

page_content=